In [45]:
import pandas
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import lightgbm as lgb 
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from numpy.random import random_sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,make_scorer
from sklearn import model_selection, preprocessing, ensemble
from sklearn import svm



In [29]:

df_test = pandas.read_json("final_train.json")
df_train = pandas.read_json("final_train.json")


categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if df_train[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df_train[f].values) + list(df_test[f].values))
            df_train[f] = lbl.transform(list(df_train[f].values))
            df_test[f] = lbl.transform(list(df_test[f].values))


In [30]:
exclude = ['price','n_no_photo','n_expensive']
feats_to_train = [x for x in df_train.columns.tolist() if df_train[x].dtype not in ['O', '<M8[ns]'] and x not in exclude]


In [31]:
train_X = df_train[feats_to_train]
train_y=df_train['price']

In [32]:
num_map = {'high':0, 'medium':1, 'low':2}
train_X = df_train[feats_to_train]
train_y = df_train['interest_level'].apply(lambda x: num_map[x])
test_X = df_test[feats_to_train]


In [38]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.33)

# https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide
# https://github.com/Microsoft/LightGBM/blob/master/docs/Python-API.md
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': {'multi_logloss'},
    'learning_rate': .05,
    'verbose': 1,
    'num_class':3,
    'estimators':20,
    'num_leaves':20
    

}


gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)






[1]	valid_0's multi_logloss: 1.06701
Train until valid scores didn't improve in 10 rounds.
[2]	valid_0's multi_logloss: 1.03792
[3]	valid_0's multi_logloss: 1.01107
[4]	valid_0's multi_logloss: 0.986447
[5]	valid_0's multi_logloss: 0.963615
[6]	valid_0's multi_logloss: 0.942337
[7]	valid_0's multi_logloss: 0.922512
[8]	valid_0's multi_logloss: 0.904263
[9]	valid_0's multi_logloss: 0.887326
[10]	valid_0's multi_logloss: 0.871455
[11]	valid_0's multi_logloss: 0.856391
[12]	valid_0's multi_logloss: 0.842496
[13]	valid_0's multi_logloss: 0.829382
[14]	valid_0's multi_logloss: 0.817013
[15]	valid_0's multi_logloss: 0.805539
[16]	valid_0's multi_logloss: 0.794764
[17]	valid_0's multi_logloss: 0.784646
[18]	valid_0's multi_logloss: 0.775218
[19]	valid_0's multi_logloss: 0.766277
[20]	valid_0's multi_logloss: 0.757918
[21]	valid_0's multi_logloss: 0.749921
[22]	valid_0's multi_logloss: 0.742541
[23]	valid_0's multi_logloss: 0.735389
[24]	valid_0's multi_logloss: 0.728819
[25]	valid_0's multi_l

In [41]:
gbm = lgb.LGBMClassifier()
gbm

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=10, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [ ]:
param_grid = {
    'learning_rate': np.arange( .01, 1, 0.1).tolist(),
    'n_estimators': sp_randint(1,100),
    'num_leaves': sp_randint(1,1024)
}



gbm_rand = RandomizedSearchCV(gbm,param_grid,cv=5,  verbose=100, scoring = 'neg_log_loss',n_iter=20)
gbm_rand.fit(train_X,train_y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] n_estimators=49, num_leaves=539, learning_rate=0.81 .............
[CV]  n_estimators=49, num_leaves=539, learning_rate=0.81, score=-0.829383 -  22.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.1s remaining:    0.0s
[CV] n_estimators=49, num_leaves=539, learning_rate=0.81 .............
[CV]  n_estimators=49, num_leaves=539, learning_rate=0.81, score=-0.848198 -  24.9s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   47.0s remaining:    0.0s
[CV] n_estimators=49, num_leaves=539, learning_rate=0.81 .............
[CV]  n_estimators=49, num_leaves=539, learning_rate=0.81, score=-0.836664 -  23.8s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min remaining:    0.0s
[CV] n_estimators=49, num_leaves=539, learning_rate=0.81 .............
[CV]  n_estimators=49, num_leaves=539, learning_rate=0.81, score=-0.848114 -  23.5s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.6min remaining:    0.0

In [28]:
print gbm_rand.best_params_,gbm_rand.best_score_
# {'n_estimators': 40, 'num_leaves': 20, 'learning_rate': 0.5} -0.567393038816


{'n_estimators': 40, 'num_leaves': 20, 'learning_rate': 0.5} -0.567393038816


In [25]:
lgb_train = lgb.Dataset(train_X, train_y)

gbm = lgb.train(params,lgb_train,early_stopping_rounds=5)


preds= gbm.predict(test_X)


In [27]:
out_df = pandas.DataFrame(preds)
out_df = pandas.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = df_test.listing_id.values


In [30]:
out_df.to_csv("lgb2.csv", index=False)